In [ ]:
from pathlib import Path
import pandas as pd
import urllib.request
from fiweps.data import d_urls
import numpy as npy

In [ ]:
PROJPATH = Path().resolve().parent

predictordir  = PROJPATH / "data/predictors"
fiwepsdir = Path("/Volumes/CWMobileSSD/DataProjects/Ongoing/preseason_predictors/incoming/teleconnections")

### Make seaice dataframe

#### From NSIDC spreadsheet

In [ ]:
bering_raw = pd.read_csv(predictordir / "N_Sea_Ice_Index_Bering_area.csv")
chukchi_raw = pd.read_csv(predictordir / "N_Sea_Ice_Index_Chukchi_area.csv")

In [ ]:
bering_raw.columns = chukchi_raw.columns = ['Year', 'January', 'February', 'March', 'April', 'May', 'June',
       'July', 'August', 'September', 'October', 'November', 'December']
bering_raw.drop(columns=['April', 'May', 'June',
       'July', 'August', 'September', 'October', 'November'], inplace=True)
bering_raw.loc[:, 'December'] = bering_raw['December'].shift(1)
chukchi_raw.drop(columns=['April', 'May', 'June',
       'July', 'August', 'September', 'October', 'November'], inplace=True)
chukchi_raw.loc[:, 'December'] = chukchi_raw['December'].shift(1)
bering_raw.set_index('Year', inplace=True)
chukchi_raw.set_index('Year', inplace=True)
chukchi_raw

,January,February,March,December
Year,,,,
1979,754104.209,781320.361,783263.263,NaN
1980,793526.278,778862.626,780134.553,781541.685
1981,791925.580,791891.445,793619.255,775716.628
1982,791229.074,756774.310,778063.746,788922.981
1983,794077.984,794850.087,799894.925,768594.157
1984,747220.491,760232.734,777401.561,721324.109
1985,790055.530,791005.084,803962.763,778720.993
1986,786220.778,803618.851,798081.620,760077.724
1987,794103.964,801511.277,804122.363,762962.522


In [ ]:
bering_raw['ber_seaice_DJ'] = bering_raw[['December', 'January']].mean(axis=1)
bering_raw['ber_seaice_DJ'] = bering_raw['ber_seaice_DJ'] - bering_raw['ber_seaice_DJ'].mean()
bering_raw['ber_seaice_FM'] = bering_raw[['February', 'March']].mean(axis=1)
bering_raw['ber_seaice_FM'] = bering_raw['ber_seaice_FM'] - bering_raw['ber_seaice_FM'].mean()
bering_raw.drop(columns=['January', 'February', 'March', 'December' ], inplace=True)
chukchi_raw['chuk_seaice_DJ'] = chukchi_raw[['December', 'January']].mean(axis=1)
chukchi_raw['chuk_seaice_DJ'] = chukchi_raw['chuk_seaice_DJ'] - chukchi_raw['chuk_seaice_DJ'].mean()
chukchi_raw['chuk_seaice_FM'] = chukchi_raw[['February', 'March']].mean(axis=1)
chukchi_raw['chuk_seaice_FM'] = chukchi_raw['chuk_seaice_FM'] - chukchi_raw['chuk_seaice_FM'].mean()
chukchi_raw.drop(columns=['January', 'February', 'March', 'December' ], inplace=True)


In [ ]:
seaice = pd.concat([bering_raw, chukchi_raw], axis=1)
seaice.columns = ['seaice_ber_anom_DJ', 'seaice_ber_anom_FM', 'seaice_chuk_anom_DJ', 'seaice_chuk_anom_FM']
# impute 1988 data from FM
seaice.loc[1988, 'seaice_ber_anom_DJ'] = seaice.loc[1988, 'seaice_ber_anom_FM']
seaice.loc[1988, 'seaice_chuk_anom_DJ'] = seaice.loc[1988, 'seaice_chuk_anom_FM']
seaice

,seaice_ber_anom_DJ,seaice_ber_anom_FM,seaice_chuk_anom_DJ,seaice_chuk_anom_FM
Year,,,,
1979,-106395.286800,-110907.192391,-11750.246811,-16483.458489
1980,72796.096700,75710.183109,21679.525689,-19276.680989
1981,86952.834700,26487.476109,17966.648189,-6019.920489
1982,40701.977200,-96388.225891,24221.571689,-31356.242489
1983,21124.388700,49359.743109,15481.614689,-1402.764489
1984,-89707.527800,94515.411609,-31582.155811,-29958.122989
1985,-72888.153800,-89663.552391,18533.805689,-1291.346989
1986,-65567.860800,-8794.728891,7294.795189,2074.965011
1987,-16733.319800,5861.737109,12678.787189,4041.549511


### Load teleconnections

#### Retrieve teleconnections to share directory

In [ ]:
outdir = Path("/Volumes/CWMobileSSD/DataProjects/Ongoing/preseason_predictors/incoming/teleconnections")

In [ ]:
d_urls.TELECONNECTIONURLS

[{'name': 'Arctic_Oscillation',
  'shortname': 'AO',
  'format': 'PSL',
  'nodata': -999,
  'skipfooter': 3,
  'URL': 'https://psl.noaa.gov/data/correlation/ao.data'},
 {'name': 'EastPac_NorthPac',
  'shortname': 'EP-NP',
  'skipfooter': 3,
  'format': 'PSL',
  'nodata': -99.9,
  'URL': 'https://psl.noaa.gov/data/correlation/epo.data'},
 {'name': 'Pacific_NA',
  'shortname': 'PNA',
  'format': 'PSL',
  'nodata': -99.9,
  'skipfooter': 3,
  'URL': 'https://psl.noaa.gov/data/correlation/pna.data'},
 {'name': 'Southern_Osc',
  'shortname': 'SOI',
  'format': 'PSL',
  'nodata': -99.99,
  'skipfooter': 3,
  'URL': 'https://psl.noaa.gov/data/correlation/soi.data'},
 {'name': 'Nino1_2',
  'shortname': 'Nino1+2',
  'format': 'PSL',
  'nodata': -99.99,
  'skipfooter': 3,
  'URL': 'https://psl.noaa.gov/data/correlation/nina1.anom.data'},
 {'name': 'Nino3',
  'shortname': 'Nino3',
  'format': 'PSL',
  'nodata': -99.99,
  'skipfooter': 3,
  'URL': 'https://psl.noaa.gov/data/correlation/nina3.anom.

In [ ]:
for item in d_urls.TELECONNECTIONURLS:
    url = item['URL']
    fn = f"{item['name']}.txt"
    print(f"retrieving {fn} from {url}")
    urllib.request.urlretrieve(url, outdir / fn)
    

retrieving Arctic_Oscillation.txt from https://psl.noaa.gov/data/correlation/ao.data
retrieving EastPac_NorthPac.txt from https://psl.noaa.gov/data/correlation/epo.data
retrieving Pacific_NA.txt from https://psl.noaa.gov/data/correlation/pna.data
retrieving Southern_Osc.txt from https://psl.noaa.gov/data/correlation/soi.data
retrieving Nino1_2.txt from https://psl.noaa.gov/data/correlation/nina1.anom.data
retrieving Nino3.txt from https://psl.noaa.gov/data/correlation/nina3.anom.data
retrieving Nino34.txt from https://psl.noaa.gov/data/correlation/nina34.anom.data
retrieving Nino4.txt from https://psl.noaa.gov/data/correlation/nina4.anom.data
retrieving Pacific_Decadal.txt from https://www.ncei.noaa.gov/pub/data/cmb/ersst/v5/index/ersst.v5.pdo.dat
retrieving Oceanic_Nino.txt from https://psl.noaa.gov/data/correlation/oni.data
retrieving North_Pac_Pattern.txt from https://climatedataguide.ucar.edu/sites/default/files/2023-04/npindex_monthly.txt
retrieving cpc_multiple_teleconn.txt from 

In [ ]:
squaredata = ['Arctic_Oscillation.txt', 'Nino3.txt', 
              'EastPac_NorthPac.txt', 'Nino34.txt', 'Southern_Osc.txt',
              'Nino1_2.txt', 'Nino4.txt', 'PacificNA.txt']
extra = 'cpc_multiple_tele_index.txt'
oni = 'Oceanic_Nino.txt'
pdo = 'Pacific_Decadal.txt'
np = 'North_Pac_Pattern.txt'

columns = ['Year'] + [str(ii) for ii in range(1, 13)]

#### Load square teleconnections

In [ ]:
squareteleconn = [teleconn for teleconn in d_urls.TELECONNECTIONURLS if teleconn['format'] == 'PSL']
squareDF = pd.DataFrame.from_records(squareteleconn)
squareDF

,name,shortname,format,nodata,skipfooter,URL
0,Arctic_Oscillation,AO,PSL,-999.00,3,https://psl.noaa.gov/data/correlation/ao.data
1,EastPac_NorthPac,EP-NP,PSL,-99.90,3,https://psl.noaa.gov/data/correlation/epo.data
2,Pacific_NA,PNA,PSL,-99.90,3,https://psl.noaa.gov/data/correlation/pna.data
3,Southern_Osc,SOI,PSL,-99.99,3,https://psl.noaa.gov/data/correlation/soi.data
4,Nino1_2,Nino1+2,PSL,-99.99,3,https://psl.noaa.gov/data/correlation/nina1.an...
5,Nino3,Nino3,PSL,-99.99,3,https://psl.noaa.gov/data/correlation/nina3.an...
6,Nino34,Nino3.4,PSL,-99.99,3,https://psl.noaa.gov/data/correlation/nina34.a...
7,Nino4,Nino4,PSL,-99.99,3,https://psl.noaa.gov/data/correlation/nina4.an...
8,Oceanic_Nino,ONI,PSL,-99.90,8,https://psl.noaa.gov/data/correlation/oni.data


In [ ]:
def get_data(row):
    fp = fiwepsdir / f"{row['name']}.txt"
    data = pd.read_csv(fp, skiprows=1, skipfooter=row.skipfooter, delim_whitespace=True, names=columns,
                       engine='python')
    data = data.astype(float)
    data['Year'] = data['Year'].astype(int)
    data.replace(row.nodata, npy.nan, inplace=True)
    data = data[data['Year'] >= 1979]
    data.meta = row['shortname']
    return data

In [ ]:
results = squareDF.apply(get_data, axis=1)
print(results[0].meta)
results[0]

AO


,Year,1,2,3,4,5,6,7,8,9,10,11,12
29,1979,-2.233,-0.697,-0.814,-1.157,-0.250,0.933,0.038,-0.684,-0.046,-1.243,0.475,1.295
30,1980,-2.066,-0.934,-1.433,-0.419,-1.155,0.722,-0.622,-0.185,0.313,-0.521,-1.361,-0.057
31,1981,-0.116,-0.332,-1.645,0.430,0.180,-0.438,0.560,-0.244,-1.040,-1.167,-0.188,-1.216
32,1982,-0.883,0.974,1.074,1.454,-0.209,-1.180,0.005,0.362,0.558,-0.211,0.661,0.967
33,1983,1.359,-1.806,-0.567,-0.738,-0.441,0.312,0.131,1.098,0.167,1.369,-0.688,0.186
34,1984,0.905,-0.303,-2.386,-0.284,0.479,0.007,0.019,0.466,-0.413,-0.270,-0.966,0.446
35,1985,-2.806,-1.440,0.551,0.652,-0.432,-0.347,-0.390,-0.001,0.114,1.035,-1.218,-1.948
36,1986,-0.568,-2.904,1.931,0.103,0.367,0.535,-0.008,-0.826,-0.023,1.425,0.926,0.060
37,1987,-1.148,-1.473,-1.747,0.387,0.325,-0.710,-0.466,-0.836,0.286,-0.080,-0.536,-0.534
38,1988,0.265,-1.066,-0.197,-0.561,-0.846,0.060,-0.143,0.255,1.039,0.032,-0.035,1.679


In [ ]:
final_results = []
for result in results:
    meta = result.meta
    result.loc[:, '12'] = result['12'].shift(1)
    result = result[result['Year'] >= 1980]
    result[f'{meta}_DJ'] = result[['12', '1']].mean(axis=1)
    result[f'{meta}_FM'] = result[['2', '3']].mean(axis=1)
    result.drop(columns=[str(ii) for ii in range(1, 13)], inplace=True)
    result.set_index('Year', inplace=True)
    final_results.append(result)
    

result_final = pd.concat(final_results, axis=1,)
result_final

/var/folders/39/6h0p929579n8vrp25cjkbz3h0000gn/T/ipykernel_51849/3026687421.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result[f'{meta}_DJ'] = result[['12', '1']].mean(axis=1)
/var/folders/39/6h0p929579n8vrp25cjkbz3h0000gn/T/ipykernel_51849/3026687421.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result[f'{meta}_FM'] = result[['2', '3']].mean(axis=1)
/var/folders/39/6h0p929579n8vrp25cjkbz3h0000gn/T/ipykernel_51849/3026687421.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy 

,AO_DJ,AO_FM,EP-NP_DJ,EP-NP_FM,PNA_DJ,PNA_FM,SOI_DJ,SOI_FM,Nino1+2_DJ,Nino1+2_FM,Nino3_DJ,Nino3_FM,Nino3.4_DJ,Nino3.4_FM,Nino4_DJ,Nino4_FM,ONI_DJ,ONI_FM
Year,,,,,,,,,,,,,,,,,,
1980,-0.3855,-1.1835,0.53,0.505,-0.845,0.695,-0.25,-0.10,-0.165,-0.215,0.405,-0.075,0.530,0.125,0.330,0.275,0.615,0.400
1981,-0.0865,-0.9885,1.02,0.000,0.945,0.740,0.25,-1.20,-0.950,-0.860,-0.350,-0.735,-0.155,-0.640,-0.080,-0.170,-0.135,-0.485
1982,-1.0495,1.0240,1.11,0.785,-1.085,-1.430,1.40,0.70,-0.220,-1.040,0.075,-0.050,-0.035,-0.020,-0.060,-0.055,-0.065,0.130
1983,1.1630,-1.1865,-0.31,-0.455,0.660,1.735,-4.75,-5.00,2.905,2.130,2.930,1.990,2.275,1.660,0.450,0.280,2.205,1.730
1984,0.5455,-1.3445,-0.97,-0.215,-0.005,0.905,0.15,0.50,-0.090,-0.875,-0.710,-0.190,-0.850,-0.355,-0.830,-0.785,-0.755,-0.380
1985,-1.1800,-0.4445,1.00,-1.540,-0.230,-1.090,-0.35,1.65,-0.590,-1.135,-1.265,-0.840,-1.250,-0.755,-0.795,-0.860,-1.090,-0.810
1986,-1.2580,-0.4865,0.52,0.450,0.840,0.460,1.00,-0.45,-0.190,-0.415,-0.765,-0.450,-0.535,-0.565,-0.245,-0.195,-0.425,-0.390
1987,-0.5440,-1.6100,-0.74,0.120,0.845,0.710,-1.75,-2.10,0.795,1.205,0.970,1.070,1.130,1.055,0.310,0.485,1.225,1.125
1988,-0.1345,-0.6315,1.13,0.065,0.265,0.875,-0.50,0.25,0.345,-0.560,0.840,-0.080,0.920,0.190,0.705,0.295,0.960,0.340


#### Load PDO and NPI

In [ ]:
fp_pdo = fiwepsdir / f"{pdo}"
data_pdo = pd.read_csv(fp_pdo, skiprows=2, delim_whitespace=True, names=columns,
                       engine='python')
data_pdo = data_pdo.astype(float)
data_pdo['Year'] = data_pdo['Year'].astype(int)
data_pdo.replace(99.99, npy.nan, inplace=True)
data_pdo.loc[:, '12'] = data_pdo['12'].shift(1)
data_pdo = data_pdo[data_pdo['Year'] >= 1980]
data_pdo.meta = 'PDO'
data_pdo[f'{meta}_DJ'] = data_pdo[['12', '1']].mean(axis=1)
data_pdo[f'{meta}_FM'] = data_pdo[['2', '3']].mean(axis=1)
data_pdo.drop(columns=[str(ii) for ii in range(1, 13)], inplace=True)
data_pdo.set_index('Year', inplace=True)


data_pdo

,ONI_DJ,ONI_FM
Year,,
1980,0.060,0.600
1981,0.740,1.230
1982,0.350,-0.280
1983,0.280,1.350
1984,1.570,1.395
1985,0.675,0.070
1986,0.685,1.575
1987,1.590,1.405
1988,0.585,0.480


Data incomplete - use Uma's NPI instead

In [ ]:
fp_npi = fiwepsdir / f"{np}"
data_npi = pd.read_csv(fp_npi, skiprows=1, delim_whitespace=True, names=['YrMnth', 'NPI'],
                       engine='python')
data_npi['Year'] = data_npi['YrMnth'].astype(str).str[:4].astype(int)
data_npi['Month'] = data_npi['YrMnth'].astype(str).str[4:].astype(int)
data_npi = data_npi[data_npi['Year'] >= 1980]
data_npi = data_npi[data_npi['Month'].isin([1, 2, 3, 4])]
data_npi = data_npi.astype(float)
data_npi['Year'] = data_npi['Year'].astype(int)
data_npi.replace(-999.0, npy.nan, inplace=True)
data_npi['label'] = data_npi.apply(
    lambda row: 'JF' if row.Month in [1, 2] else 'MA',
    axis=1
    )
data_npi.drop(columns=['YrMnth', 'Month'], inplace=True)
data_npi = data_npi.groupby(['Year', 'label']).mean().reset_index().pivot(
    index='Year', columns='label', values=['NPI'])
data_npi.columns = data_npi.columns.to_series().apply('_'.join)
data_npi



,NPI_JF,NPI_MA
Year,,
1980,1003.225,1010.930
1981,1000.895,1010.940
1982,1010.620,1016.420
1983,999.415,1009.815
1984,1007.140,1010.305
1985,1007.100,1017.305
1986,1000.980,1012.905
1987,1004.335,1010.290
1988,1004.790,1012.730


#### Add NPI from Uma

In [ ]:
fp_NPI_UB = Path("/Volumes/CWMobileSSD/DataProjects/Ongoing/preseason_predictors/incoming/teleconnections/NPI-NCEP.csv")


In [ ]:

NPI_UB = pd.read_csv(fp_NPI_UB)

NPI_UB['DT'] = pd.to_datetime(NPI_UB.Date)
NPI_UB['Year'] = NPI_UB.DT.dt.year
NPI_UB['Month'] = NPI_UB.DT.dt.month
NPI_UB = NPI_UB[NPI_UB['Month'].isin([2, 3])]
NPI_UB.drop(columns=['Total Pressure', 'Date', 'DT', 'Anomaly', 'Mean', 'Stdev'], inplace=True)
NPI_UB = NPI_UB.groupby('Year').mean()
NPI_UB.drop(columns=['Month'], inplace=True)
NPI_UB.columns=['NPI_anom_FM'] 
NPI_UB = NPI_UB[1:]
NPI_UB

,NPI_anom_FM
Year,
1980,-6.805321
1981,-6.465891
1982,5.443110
1983,-14.832285
1984,-6.559609
1985,8.899449
1986,-8.771196
1987,-4.644409
1988,-2.561996


#### Load rest of teleconnections from CPC

In [ ]:
fp_extra = fiwepsdir / f"{extra}"
with open(fp_extra, 'r') as src:
    txt = src.read().replace('-', ' -')
with open(fp_extra, 'w') as dst:
    dst.write(txt)

In [ ]:
def helper(row):
    if row.Month == 12:
        row['Year'] += 1
    return row

In [ ]:
columns = ['Year', 'Month', 'NAO', 'EA', 'WP', 'EP/NP', 'PNA', 'EA-WR', 'SCA', 'TNH', 'POL', 'PT', 'P2']
data_extra = pd.read_csv(fp_extra, skiprows=19, delim_whitespace=True, names=columns, engine='python')


data_extra = data_extra.astype(float)
data_extra['Year'] = data_extra['Year'].astype(int)
data_extra.replace(-99.9, npy.nan, inplace=True)

data_extra.apply(helper, axis=1)
data_extra = data_extra[data_extra['Year'] >= 1980]
data_extra = data_extra[data_extra['Month'].isin([1, 2, 3, 12])]
data_extra['label'] = data_extra.apply(
    lambda row: 'FM' if row.Month in [2, 3] else 'DJ',
    axis=1
    )
data_extra.drop(columns=['Month', 'EP/NP', 'PNA', 'TNH', 'PT', 'P2'], inplace=True)
data_extra = data_extra.groupby(['Year', 'label']).mean().reset_index().pivot(
    index='Year', columns='label', values=['NAO', 'EA', 'WP', 'EA-WR', 'SCA', 'POL'])
data_extra.columns = data_extra.columns.to_series().apply('_'.join)

data_extra


,NAO_DJ,NAO_FM,EA_DJ,EA_FM,WP_DJ,WP_FM,EA-WR_DJ,EA-WR_FM,SCA_DJ,SCA_FM,POL_DJ,POL_FM
Year,,,,,,,,,,,,
1980,-0.375,-0.560,-1.185,0.025,-2.060,0.295,-0.430,-0.860,-0.535,1.045,-0.675,0.235
1981,-0.095,-0.525,-0.970,0.315,-0.040,0.025,-0.755,-0.115,-0.610,-0.140,0.990,-0.330
1982,0.005,0.830,-0.785,-0.275,0.725,-0.365,0.160,1.035,-0.525,0.725,-0.035,-0.145
1983,0.760,-0.225,0.270,-0.705,-0.425,-1.015,0.165,0.220,-1.600,-0.025,-1.035,0.030
1984,0.665,-0.210,-0.745,-0.620,-0.535,-0.685,-1.000,-1.960,1.190,1.385,-0.150,0.740
1985,-1.135,-0.595,-0.330,-0.350,-0.785,0.345,-0.195,0.280,0.235,0.435,-1.405,-0.120
1986,0.805,-0.095,0.175,-0.130,-0.265,-1.565,0.155,0.640,0.325,1.045,-0.020,1.535
1987,-0.820,-0.765,0.505,-0.375,1.660,0.295,1.020,0.225,-0.600,0.255,-0.545,0.205
1988,0.580,-0.080,-0.095,-0.215,0.580,0.415,-0.170,-1.835,-0.630,0.545,0.730,-0.090


#### Concatenate full teleconnections dataframe

In [ ]:
teleconnections = pd.concat([result_final, data_pdo, NPI_UB, data_extra], axis=1,)
teleconnections

,AO_DJ,AO_FM,EP-NP_DJ,EP-NP_FM,PNA_DJ,PNA_FM,SOI_DJ,SOI_FM,Nino1+2_DJ,Nino1+2_FM,...,EA_DJ,EA_FM,WP_DJ,WP_FM,EA-WR_DJ,EA-WR_FM,SCA_DJ,SCA_FM,POL_DJ,POL_FM
Year,,,,,,,,,,,,,,,,,,,,,
1980,-0.3855,-1.1835,0.53,0.505,-0.845,0.695,-0.25,-0.10,-0.165,-0.215,...,-1.185,0.025,-2.060,0.295,-0.430,-0.860,-0.535,1.045,-0.675,0.235
1981,-0.0865,-0.9885,1.02,0.000,0.945,0.740,0.25,-1.20,-0.950,-0.860,...,-0.970,0.315,-0.040,0.025,-0.755,-0.115,-0.610,-0.140,0.990,-0.330
1982,-1.0495,1.0240,1.11,0.785,-1.085,-1.430,1.40,0.70,-0.220,-1.040,...,-0.785,-0.275,0.725,-0.365,0.160,1.035,-0.525,0.725,-0.035,-0.145
1983,1.1630,-1.1865,-0.31,-0.455,0.660,1.735,-4.75,-5.00,2.905,2.130,...,0.270,-0.705,-0.425,-1.015,0.165,0.220,-1.600,-0.025,-1.035,0.030
1984,0.5455,-1.3445,-0.97,-0.215,-0.005,0.905,0.15,0.50,-0.090,-0.875,...,-0.745,-0.620,-0.535,-0.685,-1.000,-1.960,1.190,1.385,-0.150,0.740
1985,-1.1800,-0.4445,1.00,-1.540,-0.230,-1.090,-0.35,1.65,-0.590,-1.135,...,-0.330,-0.350,-0.785,0.345,-0.195,0.280,0.235,0.435,-1.405,-0.120
1986,-1.2580,-0.4865,0.52,0.450,0.840,0.460,1.00,-0.45,-0.190,-0.415,...,0.175,-0.130,-0.265,-1.565,0.155,0.640,0.325,1.045,-0.020,1.535
1987,-0.5440,-1.6100,-0.74,0.120,0.845,0.710,-1.75,-2.10,0.795,1.205,...,0.505,-0.375,1.660,0.295,1.020,0.225,-0.600,0.255,-0.545,0.205
1988,-0.1345,-0.6315,1.13,0.065,0.265,0.875,-0.50,0.25,0.345,-0.560,...,-0.095,-0.215,0.580,0.415,-0.170,-1.835,-0.630,0.545,0.730,-0.090


April 2024: all dj and FM

In [ ]:
outfn = 'teleconnections_20240409.csv'
outpath = predictordir 

teleconnections.to_csv(outpath / outfn, float_format='%.3f')

In [ ]:
### Save out sea ice + teleconnections dataframe

In [ ]:
predictors = pd.concat([teleconnections, seaice], axis=1,).dropna()
predictors

,AO_DJ,AO_FM,EP-NP_DJ,EP-NP_FM,PNA_DJ,PNA_FM,SOI_DJ,SOI_FM,Nino1+2_DJ,Nino1+2_FM,...,EA-WR_DJ,EA-WR_FM,SCA_DJ,SCA_FM,POL_DJ,POL_FM,seaice_ber_anom_DJ,seaice_ber_anom_FM,seaice_chuk_anom_DJ,seaice_chuk_anom_FM
Year,,,,,,,,,,,,,,,,,,,,,
1980,-0.3855,-1.1835,0.53,0.505,-0.845,0.695,-0.25,-0.10,-0.165,-0.215,...,-0.430,-0.860,-0.535,1.045,-0.675,0.235,72796.096700,75710.183109,21679.525689,-19276.680989
1981,-0.0865,-0.9885,1.02,0.000,0.945,0.740,0.25,-1.20,-0.950,-0.860,...,-0.755,-0.115,-0.610,-0.140,0.990,-0.330,86952.834700,26487.476109,17966.648189,-6019.920489
1982,-1.0495,1.0240,1.11,0.785,-1.085,-1.430,1.40,0.70,-0.220,-1.040,...,0.160,1.035,-0.525,0.725,-0.035,-0.145,40701.977200,-96388.225891,24221.571689,-31356.242489
1983,1.1630,-1.1865,-0.31,-0.455,0.660,1.735,-4.75,-5.00,2.905,2.130,...,0.165,0.220,-1.600,-0.025,-1.035,0.030,21124.388700,49359.743109,15481.614689,-1402.764489
1984,0.5455,-1.3445,-0.97,-0.215,-0.005,0.905,0.15,0.50,-0.090,-0.875,...,-1.000,-1.960,1.190,1.385,-0.150,0.740,-89707.527800,94515.411609,-31582.155811,-29958.122989
1985,-1.1800,-0.4445,1.00,-1.540,-0.230,-1.090,-0.35,1.65,-0.590,-1.135,...,-0.195,0.280,0.235,0.435,-1.405,-0.120,-72888.153800,-89663.552391,18533.805689,-1291.346989
1986,-1.2580,-0.4865,0.52,0.450,0.840,0.460,1.00,-0.45,-0.190,-0.415,...,0.155,0.640,0.325,1.045,-0.020,1.535,-65567.860800,-8794.728891,7294.795189,2074.965011
1987,-0.5440,-1.6100,-0.74,0.120,0.845,0.710,-1.75,-2.10,0.795,1.205,...,1.020,0.225,-0.600,0.255,-0.545,0.205,-16733.319800,5861.737109,12678.787189,4041.549511
1988,-0.1345,-0.6315,1.13,0.065,0.265,0.875,-0.50,0.25,0.345,-0.560,...,-0.170,-1.835,-0.630,0.545,0.730,-0.090,28827.163609,28827.163609,11798.344011,11798.344011


In [ ]:
outfn = 'predictors_tele_seaice_20240409.csv'
outpath = predictordir 

predictors.to_csv(outpath / outfn, float_format='%.3f')